In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput
import io
import boto3
import sagemaker
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput


# Define the IAM role
role = get_execution_role()

# Specify the S3 bucket and prefix where your input data is stored
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sm = boto3.Session().client(service_name="sagemaker", region_name=region)
s3 = boto3.client('s3')


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
# Define the S3 bucket and prefix for training data
bucket = 'project508data'
train_data_prefix = 'train'
train_data_uri = f's3://{bucket}/{train_data_prefix}/'

# Specify the input data format and location
train_data = TrainingInput(train_data_uri, content_type='application/x-recordio')

# Define the Autopilot job configurations
auto_ml_job_config = {
    "CompletionCriteria": {
        "MaxRuntimePerTrainingJobInSeconds": 3600,
        "MaxCandidates": 250,
        "MaxAutoMLJobRuntimeInSeconds": 36000
    },
    "ProblemType": "ImageClassification",  # Specify the problem type
    "InputDataConfig": [
        {
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": train_data_uri,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ChannelName": "train",
            "ContentType": "application/x-recordio",
        }
    ],
    "JobName": "autopilot-cnn-job",
    "RoleArn": sagemaker.get_execution_role(),
    "OutputDataConfig": {
        "S3OutputPath": f's3://{bucket}/output'
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.p2.xlarge"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 3600
    },
    # Specify the number of classes and training samples
    "ProblemType": "MultiClassClassification",
    "LabelColumnName": "label",
    "NumberOfClasses": 3,  # Number of classes: normal, pneumonia, covid
    "TargetAttributeName": "label"
}

# Create a SageMaker Autopilot estimator for image classification
autopilot_estimator = sagemaker.estimator.Estimator(
    role=sagemaker.get_execution_role(),
    image_uri=sagemaker.image_uris.retrieve('image-classification', region='us-east-1'),
    instance_count=1,
    instance_type='ml.p2.xlarge',
    sagemaker_session=sagemaker.Session(),
    **auto_ml_job_config
)

# Specify a unique name for the training job
job_name = 'autopilot-cnn-job-5'

# Launch the SageMaker Autopilot job
autopilot_estimator.fit(job_name=job_name, inputs={'train': train_data}, wait=True)

In [ ]:
import sagemaker
from sagemaker import image_uris

# Specify the S3 bucket and prefix where your input data is stored
bucket = 'project508data'
prefix = 'train'

# Define the S3 URI of the parent folder containing your image data
train_data_uri = 's3://project508data/train/'
validation_data_uri = 's3://project508data/validation/'

# Create TrainingInput objects for the training and validation data
train_data = TrainingInput(train_data_uri, content_type='application/x-recordio')
validation_data = TrainingInput(validation_data_uri, content_type='application/x-recordio')

# Define the data channels
data_channels = {'train': train_data, 'validation': validation_data}

# Specify the image classification algorithm container
container = image_uris.retrieve('image-classification', region='us-east-1')

# Create an estimator for the built-in image classification algorithm
estimator = sagemaker.estimator.Estimator(
    image_uri=container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.p2.xlarge',  # Use a GPU instance type
    output_path=f's3://{bucket}/output',
    sagemaker_session=sagemaker.Session(),
    hyperparameters = {
        'num_classes': 3,
        'num_training_samples': 2700
    }
)


In [ ]:
import os
import sagemaker
import boto3
import io
from sagemaker.inputs import TrainingInput
from sagemaker.amazon.common import write_numpy_to_dense_tensor
from PIL import Image

# Specify the bucket name and prefix
bucket = 'project508data'
prefix = 'train'
recordio_prefix = 'recordio-data'

def convert_images_to_recordio(bucket_name, folder_key, buf):
    # Create a boto3 client for S3
    s3_client = boto3.client('s3')
    
    # Create a paginator to list objects in the specified S3 folder
    paginator = s3_client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=folder_key)
    
    # Iterate over the pages of objects
    for page in page_iterator:
        # Iterate over the objects in the page
        for obj in page.get('Contents', []):
            # Get the object key (filename)
            obj_key = obj['Key']
            
            # Check if the object is an image file
            if obj_key.endswith('.jpg') or obj_key.endswith('.png'):
                # Read the object data and write it to the buffer
                obj_data = s3_client.get_object(Bucket=bucket_name, Key=obj_key)['Body'].read()
                buf.write(obj_data)
    
    # Return the buffer
    return buf

bucket_name = 'project508data'
folder_key = 'train/covid'  # Replace 'covid' with the appropriate folder name
buf = io.BytesIO()
convert_images_to_recordio(bucket_name, folder_key, buf)

In [ ]:
data = buf.getvalue()

In [ ]:
data

In [ ]:
image_folders = ['covid','pneumonia', 'normal']

# Convert images to RecordIO format and upload to S3
for folder in image_folders:
    folder_path = os.path.join(prefix, folder)
    # Convert images in the folder to RecordIO format
    buf = io.BytesIO()
    convert_images_to_recordio(folder_path, buf)
    buf.seek(0)

    # Upload RecordIO data to S3
    s3_recordio_path = f's3://{bucket}/{recordio_prefix}/{folder}/data.rec'
    boto3.Session().resource('s3').Bucket(bucket).Object(s3_recordio_path).upload_fileobj(buf)

    print(f"Uploaded RecordIO data for {folder} to {s3_recordio_path}")

# Define the training data input
train_data = TrainingInput(s3_recordio_path, content_type='application/x-recordio')

# Define the data channels
data_channels = {'train': train_data}

# Pass the data channels to the fit() method of the estimator
estimator.fit(inputs=data_channels)

In [ ]:
import io
import boto3
import sagemaker
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput

# Define the S3 bucket and prefix where your input data is stored
bucket = 'project508data'
train_prefix = 'train'
validation_prefix = 'validation'

# Define the S3 URIs for train and validation data
train_data_uri = f's3://{bucket}/{train_prefix}/'
validation_data_uri = f's3://{bucket}/{validation_prefix}/'

# Specify the image classification algorithm container
container = image_uris.retrieve('image-classification', region='us-east-1')

# Function to convert images to RecordIO format
def convert_images_to_recordio(bucket_name, folder_key, buf):
    # Create a boto3 client for S3
    s3_client = boto3.client('s3')
    
    # Create a paginator to list objects in the specified S3 folder
    paginator = s3_client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=folder_key)
    
    # Iterate over the pages of objects
    for page in page_iterator:
        # Iterate over the objects in the page
        for obj in page.get('Contents', []):
            # Get the object key (filename)
            obj_key = obj['Key']
            
            # Check if the object is an image file
            if obj_key.endswith('.jpg') or obj_key.endswith('.png'):
                # Read the object data and write it to the buffer
                obj_data = s3_client.get_object(Bucket=bucket_name, Key=obj_key)['Body'].read()
                buf.write(obj_data)
    
    # Return the buffer
    return buf

# Create buffers for train and validation data
buf_train = io.BytesIO()
buf_validation = io.BytesIO()

# Convert images to RecordIO format
convert_images_to_recordio(bucket, train_prefix, buf_train)
convert_images_to_recordio(bucket, validation_prefix, buf_validation)

In [ ]:
# Create a .rec file from the buffer
buf_train.seek(0)  # Move the buffer's cursor to the beginning
rec_file_content = buf_train.getvalue()

# Specify the S3 bucket and key for the .rec file
bucket_name = 'project508data'
rec_file_key = 'train/train_data.rec'

# Upload the .rec file to S3
s3_client = boto3.client('s3')
s3_client.put_object(Body=rec_file_content, Bucket=bucket_name, Key=rec_file_key)

print(f'.rec file uploaded to s3://{bucket_name}/{rec_file_key}')

In [ ]:
# Create a .rec file from the buffer
buf_validation.seek(0)  # Move the buffer's cursor to the beginning
rec_file_content = buf_train.getvalue()

# Specify the S3 bucket and key for the .rec file
bucket_name = 'project508data'
rec_file_key = 'validation/val_data.rec'

# Upload the .rec file to S3
s3_client = boto3.client('s3')
s3_client.put_object(Body=rec_file_content, Bucket=bucket_name, Key=rec_file_key)

print(f'.rec file uploaded to s3://{bucket_name}/{rec_file_key}')

In [4]:
# Define the S3 bucket and prefix where your input data is stored
bucket = 'project508data'
train_prefix = 'train'
validation_prefix = 'validation'

# Define the S3 URIs for train and validation data
train_data_uri = f's3://{bucket}/{train_prefix}/'
validation_data_uri = f's3://{bucket}/{validation_prefix}/'

# Specify the image classification algorithm container
container = image_uris.retrieve('image-classification', region='us-east-1')

In [5]:
# Upload data from buffers to S3
s3_client = boto3.client('s3')
train_data_key = f'{train_prefix}/'
validation_data_key = f'{validation_prefix}/'

# Create TrainingInput objects
train_data_input = TrainingInput(train_data_uri, content_type='application/x-recordio')
validation_data_input = TrainingInput(validation_data_uri, content_type='application/x-recordio')

# Define the data channels
data_channels = {'train': train_data_input, 'validation': validation_data_input}

# Create an estimator for the built-in image classification algorithm
estimator = sagemaker.estimator.Estimator(
    image_uri=container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.p2.xlarge',  # Use a GPU instance type
    output_path=f's3://{bucket}/output',
    sagemaker_session=sagemaker.Session(),
    hyperparameters = {
        'num_classes': 3,
        'num_training_samples': 2700
    }
)

# Start the training job
estimator.fit(inputs=data_channels)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: image-classification-2024-04-02-02-57-35-038


2024-04-02 02:57:35 Starting - Starting the training job...
2024-04-02 02:58:00 Starting - Preparing the instances for training......
2024-04-02 02:58:38 Downloading - Downloading input data...
2024-04-02 02:59:16 Downloading - Downloading the training image.....................
2024-04-02 03:02:57 Training - Training image download completed. Training in progress.......
2024-04-02 03:04:08 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Tue Apr  2 03:03:51 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  P

UnexpectedStatusException: Error for Training job image-classification-2024-04-02-02-57-35-038: Failed. Reason: ClientError: Invalid RecordIO file in the train channel., exit code: 2

In [ ]:
import os
import mxnet as mx

def convert_images_to_recordio(image_folder, output_path):
    # Iterate over the files in the folder
    with open(output_path, 'wb') as f:
        record = mx.recordio.MXRecordIO(f)
        for root, dirs, files in os.walk(image_folder):
            for filename in files:
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    image_path = os.path.join(root, filename)
                    label = get_label_from_path(image_path)  # Implement this function to get label
                    with open(image_path, 'rb') as img_f:
                        img_bytes = img_f.read()
                        record.write(img_bytes)
                        record.write(label)

# Example usage:
image_folder = 'path_to_image_folder'
output_path = 'output.recordio'
convert_images_to_recordio(image_folder, output_path)